In [2]:
import cv2
import numpy as np
import random

GRID_SIZE = 3
TILE_SIZE = 150
WINDOW_NAME = "Sliding Puzzle"
BLANK_INDEX = GRID_SIZE * GRID_SIZE - 1

# Load and resize the image
original = cv2.imread("E:\Cv project\project\public\Assets\grayscale.jpg")
original = cv2.resize(original, (GRID_SIZE * TILE_SIZE, GRID_SIZE * TILE_SIZE))

# Slice the image into tiles
tiles = [
    original[row*TILE_SIZE:(row+1)*TILE_SIZE, col*TILE_SIZE:(col+1)*TILE_SIZE].copy()
    for row in range(GRID_SIZE)
    for col in range(GRID_SIZE)
]

# Replace the last tile with a blank
blank_tile = np.zeros_like(tiles[-1])
tiles[-1] = blank_tile

# Create a solvable shuffled state (not already solved)
def shuffle_positions():
    pos = list(range(len(tiles)))
    while True:
        random.shuffle(pos)
        if pos != list(range(len(tiles))):
            return pos

positions = shuffle_positions()

def get_tile_position(index):
    return divmod(index, GRID_SIZE)  # returns (row, col)

def draw_board():
    board = np.zeros_like(original)
    for idx, pos in enumerate(positions):
        row, col = get_tile_position(idx)
        x, y = col * TILE_SIZE, row * TILE_SIZE
        board[y:y+TILE_SIZE, x:x+TILE_SIZE] = tiles[pos]
    return board

def is_adjacent(idx1, idx2):
    r1, c1 = get_tile_position(idx1)
    r2, c2 = get_tile_position(idx2)
    return abs(r1 - r2) + abs(c1 - c2) == 1

def handle_click(event, x, y, flags, param):
    if event != cv2.EVENT_LBUTTONDOWN:
        return
    col, row = x // TILE_SIZE, y // TILE_SIZE
    clicked_idx = row * GRID_SIZE + col
    blank_idx = positions.index(BLANK_INDEX)

    if is_adjacent(blank_idx, clicked_idx):
        positions[blank_idx], positions[clicked_idx] = positions[clicked_idx], positions[blank_idx]

cv2.namedWindow(WINDOW_NAME)
cv2.setMouseCallback(WINDOW_NAME, handle_click)

# Game loop
while True:
    board = draw_board()
    cv2.imshow(WINDOW_NAME, board)

    if positions == list(range(len(tiles))):
        cv2.displayOverlay(WINDOW_NAME, "🎉 Puzzle Solved! Press ESC to exit.", 3000)

    if cv2.waitKey(30) == 27:  # ESC to quit
        break

cv2.destroyAllWindows()


In [3]:
import cv2
import numpy as np
import random

GRID_SIZE = 3
TILE_SIZE = 150
WINDOW_NAME = "Sliding Puzzle"

# Load and resize the image
original = cv2.imread("E:\Cv project\project\public\Assets\grayscale.jpg")
original = cv2.resize(original, (GRID_SIZE * TILE_SIZE, GRID_SIZE * TILE_SIZE))

# Slice into tiles
tiles = []
for row in range(GRID_SIZE):
    for col in range(GRID_SIZE):
        x, y = col * TILE_SIZE, row * TILE_SIZE
        tile = original[y:y+TILE_SIZE, x:x+TILE_SIZE].copy()
        tiles.append(tile)

# Replace last tile with a blank
blank_tile = np.zeros_like(tiles[-1])
tiles[-1] = blank_tile

# Shuffle tiles and track positions
positions = list(range(GRID_SIZE * GRID_SIZE))
random.shuffle(positions)
while not cv2.waitKey(1) == ord('s') and not any(positions[i] != i for i in range(len(positions))):
    random.shuffle(positions)  # Ensure it's not already solved on load

def get_tile_position(index):
    row = index // GRID_SIZE
    col = index % GRID_SIZE
    return row, col

def draw_board():
    board = np.zeros_like(original)
    for idx, pos in enumerate(positions):
        row, col = get_tile_position(idx)
        x, y = col * TILE_SIZE, row * TILE_SIZE
        board[y:y+TILE_SIZE, x:x+TILE_SIZE] = tiles[pos]
    return board

def is_adjacent(blank, target):
    br, bc = get_tile_position(blank)
    tr, tc = get_tile_position(target)
    return abs(br - tr) + abs(bc - tc) == 1

def handle_click(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:
        col = x // TILE_SIZE
        row = y // TILE_SIZE
        clicked_idx = row * GRID_SIZE + col
        blank_idx = positions.index(len(tiles) - 1)

        if is_adjacent(blank_idx, clicked_idx):
            positions[blank_idx], positions[clicked_idx] = positions[clicked_idx], positions[blank_idx]

cv2.namedWindow(WINDOW_NAME)
cv2.setMouseCallback(WINDOW_NAME, handle_click)

while True:
    board = draw_board()
    cv2.imshow(WINDOW_NAME, board)

    if positions == list(range(GRID_SIZE * GRID_SIZE)):
        cv2.displayOverlay(WINDOW_NAME, "🎉 You solved it! Press ESC to quit.", 3000)

    key = cv2.waitKey(100)
    if key == 27:  # ESC to quit
        break

cv2.destroyAllWindows()
